In [6]:
import os
import cv2
import src.preprocessing as ppc
from tqdm.notebook import tqdm
import json
import math

## Run this

In [7]:
with open('original_shuffled_map.json') as fp:
    original_shuffled_map = json.load(fp)

In [8]:
SUBMISSION_SET = False

if SUBMISSION_SET:
    OUT_DIR_PREFIX = 'subm_'
    CFP_DIR = './data/subm_cfp'
    IS_SHUFFLED = False
else:
    OUT_DIR_PREFIX = ''
    CFP_DIR = './data/cfp'
    IS_SHUFFLED = False

CUTOFF_THRESHOLD = 10
RESOLUTION = 224
INTERPOLATION_METHOD = cv2.INTER_CUBIC
OUT_PATH = f'data/{OUT_DIR_PREFIX}{"shuffled_" if IS_SHUFFLED else ""}square_png_{RESOLUTION}_{INTERPOLATION_METHOD}'

if not os.path.isdir(OUT_PATH):
    print(f'{OUT_PATH} does not exist, creating dir')
    os.mkdir(OUT_PATH)

data/square_png_224_2 does not exist, creating dir


In [9]:
from multiprocessing import Pool

def _make_shuffled_square_img(filename):
    try:
        file = f"{CFP_DIR}/{filename}"
        img = cv2.imread(file)
        square_img, scaling, cutting, padding = ppc.square_resize(img, RESOLUTION, CUTOFF_THRESHOLD, INTERPOLATION_METHOD)

        if IS_SHUFFLED:
            new_name = original_shuffled_map[filename] + '.png'
        else:
            new_name = filename[:-4] + '.png'
        cv2.imwrite(f"{OUT_PATH}/{new_name}", square_img)
        return (filename, new_name, cutting, padding, square_img.shape[0], scaling, img.shape)

    except Exception as e:
        print(filename, e)
        

l_files = os.listdir(CFP_DIR)

In [10]:
with Pool(8) as pool:
    op_metadata = list(tqdm(pool.imap(_make_shuffled_square_img, l_files), total=len(l_files)))    
print('Finished.')

  0%|          | 0/15000 [00:00<?, ?it/s]

Finished.


In [11]:
import pandas as pd

df_img_info = pd.DataFrame.from_records([{
        'orig_file' : opdata[0],
        'new_file' : opdata[1],
        'delta_x' : opdata[3][0] - opdata[2][0],
        'delta_y' : opdata[3][2] - opdata[2][2],
        'orig_crop_side' : round(opdata[4] * opdata[5]),
        'side' : RESOLUTION,
        'scaling' : round(opdata[4] * opdata[5]) / RESOLUTION,
    } for opdata in op_metadata if opdata is not None])

df_img_info.sort_values(by='new_file', ascending=True).to_csv(os.path.join(OUT_PATH, 'img_info.csv'), index=False)
df_img_info

,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling
0,DEV01452.jpg,DEV01452.png,-88,-904,3152,224,14.071429
1,DEV08405.jpg,DEV08405.png,171,-144,2262,224,10.098214
2,DEV10680.jpg,DEV10680.png,-138,-650,2797,224,12.486607
3,DEV13443.jpg,DEV13443.png,168,-152,2256,224,10.071429
4,DEV08466.jpg,DEV08466.png,195,-130,2334,224,10.419643
...,...,...,...,...,...,...,...
14995,DEV05006.jpg,DEV05006.png,-34,-56,1871,224,8.352679
14996,DEV04121.jpg,DEV04121.png,-131,-875,2974,224,13.276786
14997,DEV14861.jpg,DEV14861.png,-91,-624,2193,224,9.790179
14998,DEV02406.jpg,DEV02406.png,196,-128,2336,224,10.428571
